<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">TFM - Previsión de demanda mediante uso de técnicas de machine learning
</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Carlos Pérez Cebrián</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# XGBoost

XGBoost (Extreme Gradient Boosting) es una versión optimizada y escalable del algoritmo de gradient boosting, diseñada para velocidad, precisión y eficiencia. Combina múltiples modelos débiles, típicamente árboles de decisión, para crear un modelo más fuerte. XGBoost se destaca por su eficiencia computacional y rendimiento.

Es ideal para tareas que requieren grandes volúmenes de datos y alta precisión, especialmente en clasificación y regresión, utilizando ensemble learning y gradient boosting para minimizar errores.

Ensemble learning: combinación de múltiples modelos para obtener un modelo más robusto y preciso. La idea es que al combinar varios modelos, se pueden compensar los errores individuales de cada uno. 

Boosting: Se entrenan modelos secuencialmente, donde cada modelo intenta corregir los errores del modelo anterior. 

Gradient boosting es una técnica de boosting que se enfoca en minimizar los errores de predicción mediante la optimización del gradiente. Imagina que tienes un conjunto de datos y un modelo inicial que comete ciertos errores. Con cada iteración de gradient boosting, se añade un nuevo modelo que corrige esos errores, reduciendo progresivamente la cantidad de error total.


Cargamos las siguientes librerías necesarias:

In [1]:
import random

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn import cluster        # algoritmos de clustering.
from sklearn import datasets       
from sklearn.preprocessing import StandardScaler #estandarizar datos
from sklearn.decomposition import PCA #análisis de componentes principales, reducir dimensiones
from sklearn.preprocessing import LabelEncoder #codificación variables categóricas
from sklearn.model_selection import train_test_split  #split dataframes train, test, validación
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score #metricas
import xgboost as xgb
from xgboost import XGBRegressor   # Importar la clase XGBClassifier
import math

%matplotlib inline

Cargar el conjunto de datos:
(Nota: el conjunto de datos es el resultado de una fase de exploración, analisis y preparación previa.

In [2]:
# Cargar el archivo CSV con los datos de ventas
file_path = "Data/VentasRdo.csv"
file_path = "Data/VentasRdo_10productos.csv"
df = pd.read_csv(file_path, parse_dates=['idSecuencia'])

# Mostrar las primeras filas para verificar la carga
print(df.head())

   producto idSecuencia  udsVenta  bolOpen  EnPromocion  anyomes  diasemana  \
0         1  2022-12-06  0.000000        1            0        2          2   
1         2  2022-12-06  1.098612        1            0        2          2   
2         3  2022-12-06  0.000000        1            0        2          2   
3         4  2022-12-06  0.000000        1            0        2          2   
4         5  2022-12-06  0.000000        1            0        2          2   

   semana  media_7_dias  media_30_dias  venta_lag_1  venta_lag_7  venta_lag_30  
0      49      2.120606       2.301086     2.833213     2.079442           0.0  
1      49      1.874395       2.042219     2.079442     1.791759           0.0  
2      49      1.844732       1.681282     2.302585     2.564949           0.0  
3      49      1.186639       1.540582     1.098612     0.000000           0.0  
4      49      1.571729       1.871285     2.302585     2.564949           0.0  


Cargar los datos

In [3]:
# Definir las características (X) y la variable objetivo (y)
X = df[['producto', 'bolOpen', 'EnPromocion', 'diasemana', 'semana', 'media_7_dias', 'media_30_dias', 'venta_lag_1', 'venta_lag_7', 'venta_lag_30']].copy()  # Características
y = df['udsVenta']  # Variable objetivo

# Codificación de las variables categóricas
label_encoder = LabelEncoder()

# Modificar las columnas
X['producto'] = label_encoder.fit_transform(X['producto'])
X['bolOpen'] = label_encoder.fit_transform(X['bolOpen'])
X['EnPromocion'] = label_encoder.fit_transform(X['EnPromocion'])
X['diasemana'] = label_encoder.fit_transform(X['diasemana'])
X['semana'] = label_encoder.fit_transform(X['semana'])

In [4]:
# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Dimensiones de X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")

Dimensiones de X_train: (5608, 10), X_test: (1402, 10), y_train: (5608,), y_test: (1402,)


In [5]:
# Convertir los datos a un formato compatible con XGBoost (DMatrix)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Definir los parámetros del modelo XGBoost
params = {
    'objective': 'reg:squarederror',  # Para regresión (predecir un valor continuo)
    'eval_metric': 'rmse',  # Métrica de evaluación: Root Mean Squared Error (RMSE)
    'eta': 0.1,  # Tasa de aprendizaje
    'max_depth': 6,  # Profundidad máxima de los árboles
    'colsample_bytree': 0.8,  # Proporción de características a considerar por árbol
    'subsample': 0.8,  # Proporción de muestra para cada árbol
    'nthread': 4  # Número de hilos (ajustar a núcleos de CPU)
}

In [6]:
# Entrenar modelo
num_round = 100  # Número de iteraciones
modelo_xgb = xgb.train(params, dtrain, num_round)

In [7]:
# Predecir sobre test
y_pred = modelo_xgb.predict(dtest)

In [8]:
# predecir sobre train
y_train_pred = modelo_xgb.predict(dtrain)

In [9]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

RMSE: 0.7018889632416443
MSE: 0.4926481167204303
MAE: 0.5387957978632635
R²: 0.5255455479291898


In [10]:
y_pred_original = np.exp(y_pred)  # Revertir la transformación
y_test_original = np.exp(y_test)

rmse_original = mean_squared_error(y_test_original, y_pred_original, squared=False)
mse = mean_squared_error(y_test_original, y_pred_original)
mae_original = mean_absolute_error(y_test_original, y_pred_original)
r2 = r2_score(y_test_original, y_pred_original)

print(f"RMSE: {rmse}")
print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

RMSE: 0.7018889632416443
MSE: 16.307085955163657
MAE: 0.5387957978632635
R²: 0.40166687608585394


In [11]:
# guardar modelo
modelo_xgb.save_model('modelo_xgboost.json')

### otra version

In [12]:
# ajustar modelo
modelo_xgbregresion = XGBRegressor() #modelo de regresión
modelo_xgbregresion.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [13]:
y_pred = modelo_xgbregresion.predict(X_test)

In [14]:
# Métricas
rmse = mean_squared_error(y_test, y_pred, squared=False)  # RMSE
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MSE: {mse}")
print(f"MAE: {mae}")

RMSE: 0.7426122972150172
MSE: 0.5514730239749651
MAE: 0.569079576942964


In [15]:
y_pred_original = np.exp(y_pred)  # Revertir la transformación
y_test_original = np.exp(y_test)

rmse_original = mean_squared_error(y_test_original, y_pred_original, squared=False)
mse_original = mean_squared_error(y_test_original, y_pred_original)
mae_original = mean_absolute_error(y_test_original, y_pred_original)
r2_original = r2_score(y_test_original, y_pred_original)

print(f"RMSE: {rmse_original}")
print(f"MSE: {mse_original}")
print(f"MAE: {mae_original}")
print(f"R²: {r2_original}")

RMSE: 4.204137800257273
MSE: 17.674774643552066
MAE: 2.8893040452415084
R²: 0.3514841857072366


RMSE (1.99): En promedio, las predicciones están a 1.99 unidades de las reales. Este valor refleja cómo de "severa" es la desviación promedio, con penalizaciones más altas para los errores grandes.
MSE (3.96): El error cuadrado promedio es de 3.96 unidades al cuadrado. Aunque útil para ver el impacto de los errores grandes, no es directamente interpretable en las mismas unidades que la variable objetivo.
MAE (1.33): En promedio, las predicciones están a 1.33 unidades de las reales, y no penaliza los errores grandes de manera tan severa como el MSE.

Hiperparámetros más frecuentemente ajustados:

learning_rate: también llamado eta, especifica qué tan rápido el modelo ajusta los errores residuales utilizando aprendices base adicionales.
Valores típicos: 0.01–0.2

__gamma, reg_alpha, reg_lambda__: estos 3 parámetros especifican los valores para 3 tipos de regularización que realiza XGBoost: reducción mínima de la pérdida para crear una nueva división, regularización L1 en los pesos de las hojas, y regularización L2 en los pesos de las hojas, respectivamente.

Valores típicos para gamma: 0 - 0.5, pero muy dependiente de los datos. Valores típicos para reg_alpha y reg_lambda: 0 - 1 es un buen punto de partida, pero nuevamente, depende de los datos.

__max_depth__: la profundidad máxima que pueden tener los nodos de decisión del árbol. Debe ser un número entero positivo.
Valores típicos: 1–10

__subsample__: fracción del conjunto de entrenamiento que puede ser utilizada para entrenar cada árbol. Si este valor es bajo, puede llevar a un ajuste insuficiente (underfitting); si es demasiado alto, puede llevar a un sobreajuste (overfitting).
Valores típicos: 0.5–0.9

__colsample_bytree__: fracción de las características que se pueden usar para entrenar cada árbol. Un valor alto significa que casi todas las características pueden ser usadas para construir el árbol de decisión.
Valores típicos: 0.5–0.9


In [16]:
# Guardamos el modelo en un archivo .json
modelo_xgbregresion.save_model('modelo_XGBRegressor.json')